## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lafiagi,NG,8.8667,5.4167,99.75,37,28,4.41,scattered clouds
1,1,Katsuura,JP,35.1333,140.3000,69.12,78,100,9.89,overcast clouds
2,2,Mataura,NZ,-46.1927,168.8643,33.17,95,5,2.93,clear sky
3,3,Belmonte,BR,-15.8631,-38.8828,84.70,64,70,11.88,broken clouds
4,4,Cidreira,BR,-30.1811,-50.2056,55.06,73,90,23.73,overcast clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Belmonte,BR,-15.8631,-38.8828,84.70,64,70,11.88,broken clouds
7,7,Atuona,PF,-9.8000,-139.0333,80.01,76,32,18.81,scattered clouds
9,9,Hithadhoo,MV,-0.6000,73.0833,84.74,71,100,15.19,overcast clouds
12,12,Padang,ID,-0.9492,100.3543,78.85,86,86,3.00,overcast clouds
15,15,Nabire,ID,-3.3667,135.4833,76.42,90,97,3.40,overcast clouds
29,29,Petatlan,MX,17.5167,-101.2667,76.80,42,3,1.03,clear sky
33,33,Saint-Georges,GD,12.0564,-61.7485,83.88,74,75,24.16,broken clouds
38,38,Mahibadhoo,MV,3.7833,72.9667,83.75,75,20,18.88,few clouds
40,40,Faya,SA,18.3851,42.4509,78.91,25,25,11.23,scattered clouds
41,41,Arraial Do Cabo,BR,-22.9661,-42.0278,78.76,61,75,8.05,broken clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
clean_df = preferred_cities_df.dropna()


In [12]:
clean_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Belmonte,BR,84.70,broken clouds,-15.8631,-38.8828,
7,Atuona,PF,80.01,scattered clouds,-9.8000,-139.0333,
9,Hithadhoo,MV,84.74,overcast clouds,-0.6000,73.0833,
12,Padang,ID,78.85,overcast clouds,-0.9492,100.3543,
15,Nabire,ID,76.42,overcast clouds,-3.3667,135.4833,
29,Petatlan,MX,76.80,clear sky,17.5167,-101.2667,
33,Saint-Georges,GD,83.88,broken clouds,12.0564,-61.7485,
38,Mahibadhoo,MV,83.75,few clouds,3.7833,72.9667,
40,Faya,SA,78.91,scattered clouds,18.3851,42.4509,
41,Arraial Do Cabo,BR,78.76,broken clouds,-22.9661,-42.0278,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Belmonte,BR,84.70,broken clouds,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
7,Atuona,PF,80.01,scattered clouds,-9.8000,-139.0333,Villa Enata
9,Hithadhoo,MV,84.74,overcast clouds,-0.6000,73.0833,Scoop Guest House
12,Padang,ID,78.85,overcast clouds,-0.9492,100.3543,Mercure Padang
15,Nabire,ID,76.42,overcast clouds,-3.3667,135.4833,Hotel Nusantara Nabire
29,Petatlan,MX,76.80,clear sky,17.5167,-101.2667,Hotel California Petatlán
33,Saint-Georges,GD,83.88,broken clouds,12.0564,-61.7485,Radisson Grenada Beach Resort
38,Mahibadhoo,MV,83.75,few clouds,3.7833,72.9667,Omadhoo Inn
40,Faya,SA,78.91,scattered clouds,18.3851,42.4509,مقبرة ال مجمل
41,Arraial Do Cabo,BR,78.76,broken clouds,-22.9661,-42.0278,Pousada Porto Praia


In [16]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

In [17]:
hotel_df.isnull().sum()

City                    0
Country                 0
Max Temp                0
Current Description     0
Lat                     0
Lng                     0
Hotel Name             14
dtype: int64

In [18]:
hotel_df.dropna(subset=["Hotel Name"], inplace=True)

In [19]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [20]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Belmonte,BR,84.70,broken clouds,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
7,Atuona,PF,80.01,scattered clouds,-9.8000,-139.0333,Villa Enata
9,Hithadhoo,MV,84.74,overcast clouds,-0.6000,73.0833,Scoop Guest House
12,Padang,ID,78.85,overcast clouds,-0.9492,100.3543,Mercure Padang
15,Nabire,ID,76.42,overcast clouds,-3.3667,135.4833,Hotel Nusantara Nabire
29,Petatlan,MX,76.80,clear sky,17.5167,-101.2667,Hotel California Petatlán
33,Saint-Georges,GD,83.88,broken clouds,12.0564,-61.7485,Radisson Grenada Beach Resort
38,Mahibadhoo,MV,83.75,few clouds,3.7833,72.9667,Omadhoo Inn
40,Faya,SA,78.91,scattered clouds,18.3851,42.4509,مقبرة ال مجمل
41,Arraial Do Cabo,BR,78.76,broken clouds,-22.9661,-42.0278,Pousada Porto Praia


In [21]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
#hotel_df.to_csv(r"C:\Users\laure\OneDrive\Desktop\Class Files\World_Weather_Analysis\Weather_Database\Vacation_Search\WeatherPy_Vacation.csv", index_label="City_ID")
hotel_df.to_csv("WeatherPy_Vacation.csv", index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
# info_box_template = """
# <dl>
# <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# <dt>Max Temp</dt><dd>{Max Temp} °F</dd>
# </dl>
# """

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
locations = hotel_df[["Lat", "Lng"]]
#max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
            # max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))